<a href="https://colab.research.google.com/github/logankusuma/goodreads_scrape/blob/main/Scrape_Goodreads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import tools tambahan
from urllib.request import urlopen      #Akses ke halaman website
import re
from bs4 import BeautifulSoup           #Mengubah halaman website ke bentuk text
import pandas as pd                     #Menyusun teks menjadi data frame
from google.colab import files          #mengubah data menjadi file
import datetime

In [ ]:
#Jumlah halaman
npage = 6

books= []
loop = 0
#Loop halaman 1-n
for p in range(npage):
  url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever?page=' + str(p+1)+ '&ref=ls_pl_car_0'                             #link ke good reads
  #url = 'https://www.goodreads.com/list/show/1572.Buku_Indonesia_Sepanjang_Masa?page=' + str(p+1)+ '&ref=ls_pl_car_0'

  goodreads = urlopen(url)
  goodreads_content = goodreads.read()

  goodreads_html = BeautifulSoup(goodreads_content, 'html.parser')                                #Baca html sebagai text

  #print(goodreads_html)

  books_tr = goodreads_html.find_all("tr",attrs={'itemtype':'http://schema.org/Book'})
  for tr in books_tr:
    #loop += 1
    #print(loop)
    books_rank = tr.find("td", attrs={'class':'number'}).text
    books_title = tr.find("a", attrs={'class':'bookTitle'}).span.text
    books_author = tr.find("a", attrs={'class':'authorName'}).span.text
    books_rating_raw = tr.find("span", attrs={'class':'minirating'}).text
    books_score_raw = tr.find(style="margin-top: 5px").span.a.text

    books_end_link = tr.find("a", attrs={'class':'bookTitle'})['href']
    books_link = 'https://www.goodreads.com' + books_end_link

    books_url = urlopen(books_link)
    books_url_content = books_url.read()

    #Scrape data per buku
    books_html = BeautifulSoup(books_url_content, 'html.parser')

    try:
      books_desc = books_html.find("div", attrs={'data-testid':"description"}).div.div.div.div.span.text
    except:
      books_desc = float("NaN")

    try:
      books_genre_span = books_html.find_all("span",attrs={'class':'BookPageMetadataSection__genreButton'})
      books_genre_list = []
      for gen in books_genre_span:
        temp = gen.a.span.text
        books_genre_list.append({"Genre_list":temp})
    except:
      books_genre_list = float("NaN")

    try:
      books_page_raw = books_html.find("p", attrs={'data-testid':"pagesFormat"}).text
    except:
      books_page_raw = float("NaN")
    try:
      books_pub_raw = books_html.find("p", attrs={'data-testid':"publicationInfo"}).text
    except:
      books_pub_raw = float("NaN")

    try:
      books_cread_raw = books_html.find("div", attrs={'data-testid':"currentlyReadingSignal"}).text
    except:
      books_cread_raw = float("NaN")
    try:
      books_wread_raw = books_html.find("div", attrs={'data-testid':"toReadSignal"}).text
    except:
      books_wread_raw = float("NaN")

    try:
      div1 = books_html.find("div", attrs={'class':"FeaturedPerson"}).div
      div2 = div1.find_all("div")
      div3 = div2[1].div.div
      span1 = div3.find("span", attrs={'class':"Text"})
      auth_stats_raw = span1.text
    except:
      auth_stats_raw = float("NaN")

    try:
      rating5_raw = books_html.find("div", attrs={'data-testid':"labelTotal-5"}).text
    except:
      rating5_raw = float("NaN")
    try:
      rating4_raw = books_html.find("div", attrs={'data-testid':"labelTotal-4"}).text
    except:
      rating4_raw = float("NaN")
    try:
      rating3_raw = books_html.find("div", attrs={'data-testid':"labelTotal-3"}).text
    except:
      rating3_raw = float("NaN")
    try:
      rating2_raw = books_html.find("div", attrs={'data-testid':"labelTotal-2"}).text
    except:
      rating2_raw = float("NaN")
    try:
      rating1_raw = books_html.find("div", attrs={'data-testid':"labelTotal-1"}).text
    except:
      rating1_raw = float("NaN")

    books.append({"Rank" : books_rank, "Title" : books_title,
                  "Author" : books_author, "Rating_raw" : books_rating_raw,
                  "Score_raw": books_score_raw, "Link" : books_link,
                  "Description": books_desc, "Genre_list": books_genre_list,
                  "Page_raw": books_page_raw, "Publication_raw": books_pub_raw,
                  "CurrentlyRead_raw": books_cread_raw, "toRead_raw" : books_wread_raw,
                  "AuthorStats_raw": auth_stats_raw, "Rating5_raw":rating5_raw,
                  "Rating4_raw":rating4_raw,"Rating3_raw":rating3_raw,
                  "Rating2_raw":rating2_raw,"Rating1_raw":rating1_raw,})

print(books)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
books_df = pd.DataFrame(books)
print(books_df)

    Rank                                    Title                 Author  \
0      1                           Laskar Pelangi          Andrea Hirata   
1      2        Bumi Manusia (Tetralogi Buru, #1)  Pramoedya Ananta Toer   
2      3                                     5 cm      Donny Dhirgantoro   
3      4                     Ronggeng Dukuh Paruk           Ahmad Tohari   
4      5                          Negeri 5 Menara            Ahmad Fuadi   
..   ...                                      ...                    ...   
522  523                  Pengurus MOS harus Mati               Lexie Xu   
523  524                       Surat Kertas Hijau       Sitor Situmorang   
524  525                            Kemelut Hidup          Ramadhan K.H.   
525  526  Anjing Penjaga: Pers Di Rumah Orde Baru        Omi Intan Naomi   
526  526     The Power of Word of Mouth Marketing                Sumardy   

                                       Rating_raw      Score_raw  \
0                4.

In [ ]:
books_df.to_csv("bestbook.csv", index=False)
files.download("bestbook.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>